In [1]:
from __future__ import print_function
import torch
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import torch.utils.data
import torch.nn.init as init
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import VAE

print( 'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

torch: 1.3.0+cpu cuda: False


## Load & Re-Scale Data Sets

In [2]:
path = "train_2_4000Samples.csv"
train_data = pd.read_csv(path).iloc[:,1:]
test_data = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_2_800Samples.csv')).iloc[:,1:]
train_data = train_data.iloc[:,:-1]
test_data = test_data.iloc[:,:-1]
train_data.columns  = test_data.columns
cols = test_data.columns
scalar = MinMaxScaler().fit(pd.concat([train_data, test_data]))
train_data = pd.DataFrame(scalar.transform (train_data)) 
test_data = pd.DataFrame(scalar.transform (test_data)) 
train_data.columns = cols
test_data.columns = cols

## Fix the Global Variables for Learning

In [3]:
#hyperparameters
x_dim = train_data.shape[1]
h_dim = 350
z_dim = 100
n_epochs = 100
clip = 10
learning_rate = 1e-3
batch_size = 25
seed = 100
print_every = 10
save_every = 10
#manual seed
torch.manual_seed(seed)
#init model + optimizer + datasets
train_loader = torch.utils.data.DataLoader ( dataset = train_data.values ,  batch_size = batch_size , shuffle= True)
test_loader = torch.utils.data.DataLoader (  dataset = test_data.values , shuffle= True)
model = VAE.VAE(x_dim, h_dim, z_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Train and Test the VAE

In [4]:
def train(epoch):
    train_loss = 0
    epoch_loss = np.zeros(int(len (train_data) / batch_size ))
    epoch_div = np.zeros(int(len (train_data) / batch_size))
    for batch_idx, (data) in enumerate(train_loader):
        
        data = Variable(data)
        #forward + backward + optimize
        optimizer.zero_grad()
        kld_loss, nll_loss, lat, recon, _ = model(data)
        epoch_loss [batch_idx] = nll_loss
        epoch_div [batch_idx] = kld_loss
        loss = kld_loss + nll_loss
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        #printing
        if batch_idx % print_every == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t KLD Loss: {:.6f} \t NLL Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                kld_loss.data / batch_size,
                nll_loss.data / batch_size))

            

        train_loss += loss.data
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))
    return epoch_loss, epoch_div
    
def test(epoch):
    """uses test data to evaluate 
    likelihood of the model"""
    mean_kld_loss, mean_nll_loss = 0, 0
    epoch_loss = np.zeros(len(test_data))
    epoch_div = np.zeros(len(test_data))
    for i, (data) in enumerate(test_loader):                                           
        
        data = Variable(data.reshape(1,-1))
        kld_loss, nll_loss, _, _, _ = model(data)
        epoch_div [i] = kld_loss
        epoch_loss [i] = nll_loss
        mean_kld_loss += kld_loss.data
        mean_nll_loss += nll_loss.data

    mean_kld_loss /= len(test_loader.dataset)
    mean_nll_loss /= len(test_loader.dataset)

    print('====> Test set loss: KLD Loss = {:.4f}, NLL Loss = {:.4f} '.format(
        mean_kld_loss, mean_nll_loss))
    return epoch_loss, epoch_div

## Train and Test the Model

In [5]:
train_error = np.zeros([n_epochs , int(train_data.shape[0] / batch_size ) ])
train_div = np.zeros([n_epochs , int(train_data.shape[0] / batch_size ) ])
test_error , test_div  = np.zeros([n_epochs , test_data.shape[0]]) , np.zeros([n_epochs , test_data.shape[0]]) 
for epoch in range(1, n_epochs + 1):
    #training + testing
    tr = train(epoch)
    train_error [epoch-1 , :] = tr [0]
    train_div [epoch-1 , :] = tr [1] 
    te = test(epoch)
    test_error [epoch-1 , :] = te [0]
    test_div [epoch-1 , :] = te [1]

Train Epoch: 1 [0/4000 (0%)]	 KLD Loss: 38.020260 	 NLL Loss: 227.864865
Train Epoch: 1 [250/4000 (6%)]	 KLD Loss: 22.897991 	 NLL Loss: 112.060410
Train Epoch: 1 [500/4000 (12%)]	 KLD Loss: 12.237794 	 NLL Loss: 51.085299
Train Epoch: 1 [750/4000 (19%)]	 KLD Loss: 6.369750 	 NLL Loss: 43.438351
Train Epoch: 1 [1000/4000 (25%)]	 KLD Loss: 3.470811 	 NLL Loss: 39.820648
Train Epoch: 1 [1250/4000 (31%)]	 KLD Loss: 3.072402 	 NLL Loss: 37.801216
Train Epoch: 1 [1500/4000 (38%)]	 KLD Loss: 3.427801 	 NLL Loss: 55.838341
Train Epoch: 1 [1750/4000 (44%)]	 KLD Loss: 2.368503 	 NLL Loss: 40.998783
Train Epoch: 1 [2000/4000 (50%)]	 KLD Loss: 2.483226 	 NLL Loss: 38.798042
Train Epoch: 1 [2250/4000 (56%)]	 KLD Loss: 4.403807 	 NLL Loss: 80.071478
Train Epoch: 1 [2500/4000 (62%)]	 KLD Loss: 3.459243 	 NLL Loss: 38.467321
Train Epoch: 1 [2750/4000 (69%)]	 KLD Loss: 2.062087 	 NLL Loss: 54.412907
Train Epoch: 1 [3000/4000 (75%)]	 KLD Loss: 3.126553 	 NLL Loss: 37.549738
Train Epoch: 1 [3250/4000 (8

Train Epoch: 13 [2000/4000 (50%)]	 KLD Loss: 3.351301 	 NLL Loss: -105.097459
Train Epoch: 13 [2250/4000 (56%)]	 KLD Loss: 3.267094 	 NLL Loss: -98.568546
Train Epoch: 13 [2500/4000 (62%)]	 KLD Loss: 3.230361 	 NLL Loss: -95.221392
Train Epoch: 13 [2750/4000 (69%)]	 KLD Loss: 3.308179 	 NLL Loss: -101.285199
Train Epoch: 13 [3000/4000 (75%)]	 KLD Loss: 3.347207 	 NLL Loss: -100.415154
Train Epoch: 13 [3250/4000 (81%)]	 KLD Loss: 3.325738 	 NLL Loss: -95.177127
Train Epoch: 13 [3500/4000 (88%)]	 KLD Loss: 3.290824 	 NLL Loss: -95.990781
Train Epoch: 13 [3750/4000 (94%)]	 KLD Loss: 3.405978 	 NLL Loss: -90.708348
====> Epoch: 13 Average loss: -94.0685
====> Test set loss: KLD Loss = 3.2681, NLL Loss = -94.6773 
Train Epoch: 14 [0/4000 (0%)]	 KLD Loss: 3.329579 	 NLL Loss: -105.619846
Train Epoch: 14 [250/4000 (6%)]	 KLD Loss: 3.401813 	 NLL Loss: -98.509664
Train Epoch: 14 [500/4000 (12%)]	 KLD Loss: 3.341558 	 NLL Loss: -105.137834
Train Epoch: 14 [750/4000 (19%)]	 KLD Loss: 3.336873 	 

Train Epoch: 25 [2500/4000 (62%)]	 KLD Loss: 3.403638 	 NLL Loss: -142.116490
Train Epoch: 25 [2750/4000 (69%)]	 KLD Loss: 3.559971 	 NLL Loss: -145.846869
Train Epoch: 25 [3000/4000 (75%)]	 KLD Loss: 3.587150 	 NLL Loss: -144.948417
Train Epoch: 25 [3250/4000 (81%)]	 KLD Loss: 3.586895 	 NLL Loss: -153.707069
Train Epoch: 25 [3500/4000 (88%)]	 KLD Loss: 3.411478 	 NLL Loss: -138.228140
Train Epoch: 25 [3750/4000 (94%)]	 KLD Loss: 3.481547 	 NLL Loss: -148.106356
====> Epoch: 25 Average loss: -141.4536
====> Test set loss: KLD Loss = 3.4830, NLL Loss = -143.6968 
Train Epoch: 26 [0/4000 (0%)]	 KLD Loss: 3.422776 	 NLL Loss: -151.291952
Train Epoch: 26 [250/4000 (6%)]	 KLD Loss: 3.454670 	 NLL Loss: -145.177253
Train Epoch: 26 [500/4000 (12%)]	 KLD Loss: 3.500671 	 NLL Loss: -157.637819
Train Epoch: 26 [750/4000 (19%)]	 KLD Loss: 3.413907 	 NLL Loss: -155.391225
Train Epoch: 26 [1000/4000 (25%)]	 KLD Loss: 3.449323 	 NLL Loss: -149.695244
Train Epoch: 26 [1250/4000 (31%)]	 KLD Loss: 3.4

Train Epoch: 37 [3000/4000 (75%)]	 KLD Loss: 3.421950 	 NLL Loss: -175.212284
Train Epoch: 37 [3250/4000 (81%)]	 KLD Loss: 3.375096 	 NLL Loss: -169.888171
Train Epoch: 37 [3500/4000 (88%)]	 KLD Loss: 3.398513 	 NLL Loss: -170.691250
Train Epoch: 37 [3750/4000 (94%)]	 KLD Loss: 3.424877 	 NLL Loss: -175.107649
====> Epoch: 37 Average loss: -167.5761
====> Test set loss: KLD Loss = 3.4450, NLL Loss = -162.7090 
Train Epoch: 38 [0/4000 (0%)]	 KLD Loss: 3.410666 	 NLL Loss: -169.483277
Train Epoch: 38 [250/4000 (6%)]	 KLD Loss: 3.355415 	 NLL Loss: -168.898698
Train Epoch: 38 [500/4000 (12%)]	 KLD Loss: 3.420197 	 NLL Loss: -175.848099
Train Epoch: 38 [750/4000 (19%)]	 KLD Loss: 3.456096 	 NLL Loss: -172.679896
Train Epoch: 38 [1000/4000 (25%)]	 KLD Loss: 3.465544 	 NLL Loss: -164.798472
Train Epoch: 38 [1250/4000 (31%)]	 KLD Loss: 3.434395 	 NLL Loss: -170.289735
Train Epoch: 38 [1500/4000 (38%)]	 KLD Loss: 3.566706 	 NLL Loss: -165.551527
Train Epoch: 38 [1750/4000 (44%)]	 KLD Loss: 3.4

Train Epoch: 49 [3500/4000 (88%)]	 KLD Loss: 3.416976 	 NLL Loss: -175.562627
Train Epoch: 49 [3750/4000 (94%)]	 KLD Loss: 3.459720 	 NLL Loss: -164.971117
====> Epoch: 49 Average loss: -175.5720
====> Test set loss: KLD Loss = 3.4466, NLL Loss = -176.3303 
Train Epoch: 50 [0/4000 (0%)]	 KLD Loss: 3.459992 	 NLL Loss: -182.570529
Train Epoch: 50 [250/4000 (6%)]	 KLD Loss: 3.380810 	 NLL Loss: -176.528799
Train Epoch: 50 [500/4000 (12%)]	 KLD Loss: 3.309403 	 NLL Loss: -182.038141
Train Epoch: 50 [750/4000 (19%)]	 KLD Loss: 3.483855 	 NLL Loss: -180.877674
Train Epoch: 50 [1000/4000 (25%)]	 KLD Loss: 3.393607 	 NLL Loss: -168.541540
Train Epoch: 50 [1250/4000 (31%)]	 KLD Loss: 3.427586 	 NLL Loss: -188.960239
Train Epoch: 50 [1500/4000 (38%)]	 KLD Loss: 3.529935 	 NLL Loss: -180.313753
Train Epoch: 50 [1750/4000 (44%)]	 KLD Loss: 3.384773 	 NLL Loss: -187.013639
Train Epoch: 50 [2000/4000 (50%)]	 KLD Loss: 3.431467 	 NLL Loss: -179.199402
Train Epoch: 50 [2250/4000 (56%)]	 KLD Loss: 3.5

====> Test set loss: KLD Loss = 3.5139, NLL Loss = -174.6191 
Train Epoch: 62 [0/4000 (0%)]	 KLD Loss: 3.510786 	 NLL Loss: -184.018026
Train Epoch: 62 [250/4000 (6%)]	 KLD Loss: 3.483943 	 NLL Loss: -190.612812
Train Epoch: 62 [500/4000 (12%)]	 KLD Loss: 3.310579 	 NLL Loss: -195.531793
Train Epoch: 62 [750/4000 (19%)]	 KLD Loss: 3.625553 	 NLL Loss: -190.759085
Train Epoch: 62 [1000/4000 (25%)]	 KLD Loss: 3.391104 	 NLL Loss: -189.836676
Train Epoch: 62 [1250/4000 (31%)]	 KLD Loss: 3.522699 	 NLL Loss: -188.803066
Train Epoch: 62 [1500/4000 (38%)]	 KLD Loss: 3.439088 	 NLL Loss: -183.920931
Train Epoch: 62 [1750/4000 (44%)]	 KLD Loss: 3.378662 	 NLL Loss: -196.122803
Train Epoch: 62 [2000/4000 (50%)]	 KLD Loss: 3.494692 	 NLL Loss: -181.110222
Train Epoch: 62 [2250/4000 (56%)]	 KLD Loss: 3.489188 	 NLL Loss: -183.819630
Train Epoch: 62 [2500/4000 (62%)]	 KLD Loss: 3.569098 	 NLL Loss: -182.804234
Train Epoch: 62 [2750/4000 (69%)]	 KLD Loss: 3.472096 	 NLL Loss: -179.536936
Train Epoc

Train Epoch: 74 [500/4000 (12%)]	 KLD Loss: 3.560532 	 NLL Loss: -189.756016
Train Epoch: 74 [750/4000 (19%)]	 KLD Loss: 3.491585 	 NLL Loss: -189.778515
Train Epoch: 74 [1000/4000 (25%)]	 KLD Loss: 3.625743 	 NLL Loss: -185.285541
Train Epoch: 74 [1250/4000 (31%)]	 KLD Loss: 3.515198 	 NLL Loss: -181.888797
Train Epoch: 74 [1500/4000 (38%)]	 KLD Loss: 3.498646 	 NLL Loss: -192.470006
Train Epoch: 74 [1750/4000 (44%)]	 KLD Loss: 3.541317 	 NLL Loss: -192.700053
Train Epoch: 74 [2000/4000 (50%)]	 KLD Loss: 3.578253 	 NLL Loss: -191.101305
Train Epoch: 74 [2250/4000 (56%)]	 KLD Loss: 3.585705 	 NLL Loss: -191.342428
Train Epoch: 74 [2500/4000 (62%)]	 KLD Loss: 3.516226 	 NLL Loss: -189.193841
Train Epoch: 74 [2750/4000 (69%)]	 KLD Loss: 3.570700 	 NLL Loss: -191.574418
Train Epoch: 74 [3000/4000 (75%)]	 KLD Loss: 3.560886 	 NLL Loss: -191.982356
Train Epoch: 74 [3250/4000 (81%)]	 KLD Loss: 3.513677 	 NLL Loss: -197.783096
Train Epoch: 74 [3500/4000 (88%)]	 KLD Loss: 3.572501 	 NLL Loss: 

Train Epoch: 86 [1000/4000 (25%)]	 KLD Loss: 3.646447 	 NLL Loss: -193.238042
Train Epoch: 86 [1250/4000 (31%)]	 KLD Loss: 3.704545 	 NLL Loss: -194.078472
Train Epoch: 86 [1500/4000 (38%)]	 KLD Loss: 3.619137 	 NLL Loss: -198.798934
Train Epoch: 86 [1750/4000 (44%)]	 KLD Loss: 3.478817 	 NLL Loss: -200.803174
Train Epoch: 86 [2000/4000 (50%)]	 KLD Loss: 3.602751 	 NLL Loss: -185.691411
Train Epoch: 86 [2250/4000 (56%)]	 KLD Loss: 3.570113 	 NLL Loss: -196.830566
Train Epoch: 86 [2500/4000 (62%)]	 KLD Loss: 3.489019 	 NLL Loss: -190.397080
Train Epoch: 86 [2750/4000 (69%)]	 KLD Loss: 3.639992 	 NLL Loss: -191.663415
Train Epoch: 86 [3000/4000 (75%)]	 KLD Loss: 3.558748 	 NLL Loss: -193.873437
Train Epoch: 86 [3250/4000 (81%)]	 KLD Loss: 3.543249 	 NLL Loss: -198.990626
Train Epoch: 86 [3500/4000 (88%)]	 KLD Loss: 3.671430 	 NLL Loss: -192.837397
Train Epoch: 86 [3750/4000 (94%)]	 KLD Loss: 3.704529 	 NLL Loss: -182.879024
====> Epoch: 86 Average loss: -190.5111
====> Test set loss: KLD

Train Epoch: 98 [1500/4000 (38%)]	 KLD Loss: 3.639830 	 NLL Loss: -204.126354
Train Epoch: 98 [1750/4000 (44%)]	 KLD Loss: 3.565673 	 NLL Loss: -201.530673
Train Epoch: 98 [2000/4000 (50%)]	 KLD Loss: 3.796359 	 NLL Loss: -196.896320
Train Epoch: 98 [2250/4000 (56%)]	 KLD Loss: 3.631363 	 NLL Loss: -197.000576
Train Epoch: 98 [2500/4000 (62%)]	 KLD Loss: 3.517593 	 NLL Loss: -205.185613
Train Epoch: 98 [2750/4000 (69%)]	 KLD Loss: 3.607688 	 NLL Loss: -198.902982
Train Epoch: 98 [3000/4000 (75%)]	 KLD Loss: 3.730620 	 NLL Loss: -199.196332
Train Epoch: 98 [3250/4000 (81%)]	 KLD Loss: 3.676972 	 NLL Loss: -195.115222
Train Epoch: 98 [3500/4000 (88%)]	 KLD Loss: 3.723382 	 NLL Loss: -200.687855
Train Epoch: 98 [3750/4000 (94%)]	 KLD Loss: 3.564331 	 NLL Loss: -198.622327
====> Epoch: 98 Average loss: -194.4795
====> Test set loss: KLD Loss = 3.6603, NLL Loss = -198.0704 
Train Epoch: 99 [0/4000 (0%)]	 KLD Loss: 3.646936 	 NLL Loss: -208.610844
Train Epoch: 99 [250/4000 (6%)]	 KLD Loss: 3

## Get Latent Representations and Save the Model for later Use

In [6]:
train_lat = [ model (Variable(torch.tensor(train_data.iloc[idx,:].values)).reshape(1,-1))[-1] for idx in range(len(train_data)) ]
test_lat = [ model (Variable(torch.tensor(test_data.iloc[idx,:].values)).reshape(1,-1))[-1] for idx in range(len(test_data)) ]
train_lat = pd.DataFrame(torch.cat(train_lat).cpu().detach().numpy())
test_lat = pd.DataFrame(torch.cat(test_lat).cpu().detach().numpy())
cols = []
for i in range(train_lat.shape[1]):
    cols.append(str('Z'+str(i+1)))
train_lat.columns = cols
test_lat.columns = cols
train_lat.to_csv('Training_Data_Sets\\latent_200D.csv')
test_lat.to_csv('Test_Data_Sets\\latent_200D.csv')
with open('Models\\200D.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
array = np.zeros((10000,100))
for i in range(len(array)):
    array [i] = model(Variable(torch.tensor(np.zeros(200))))[-1].cpu().detach().numpy()
array1 = np.zeros((10000,100))
for i in range(len(array)):
    array1 [i] = model(Variable(torch.tensor(np.ones(200))))[-1].cpu().detach().numpy()

In [8]:
np.mean(abs(np.mean(array1, 0)-np.mean(array, 0)))/4

0.1566458388853974